https://colab.research.google.com/drive/12Pw4-KfEESaCi0osmi65CPfxoafuJ1ik?usp=sharing

# Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

from keras.layers import Dense
from keras.models import Model

from sklearn import metrics

import random

import os

# Импорт данных

In [2]:
# установка Kaggle Api
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list

In [6]:
!kaggle competitions download -c dogs-vs-cats

In [7]:
!mkdir train
!unzip train.zip -d train

!mkdir test
!unzip test1.zip -d test

In [8]:
!rm train.zip
!rm test1.zip

# Предобработка данных

In [9]:
# вспомогательная функция
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x  

In [10]:
# получение путей изображений для обучающей выборки
train_dir = os.path.join('/content', 'train', 'train')
train_images = os.listdir(train_dir)

Нейросеть будет обучаться на 2500 изображениях, поскольку не хватит мощности на обработку 25000 изображений

In [11]:
# названия изображений обучающей выборки
train_images = train_images[:2500]

In [12]:
# формирование датафрейма
categories = list()
for img in train_images:
    category = img.split(".")[0]
    if category == "dog":
        categories.append("1")
    else:
        categories.append("0")
train_df = pd.DataFrame({"Image":train_images, "Category": categories})

In [13]:
# как выглядит датафрейм
train_df.head()

,Image,Category
0,dog.5577.jpg,1
1,dog.8301.jpg,1
2,cat.11000.jpg,0
3,dog.8878.jpg,1
4,dog.11820.jpg,1


In [14]:
# формирование обучающей выборки
train = []
for img_name, category in zip(train_df['Image'], train_df['Category']):
  img, x = get_image(os.path.join(train_dir, img_name))
  train.append({'x': np.array(x[0]), 'y': category })

In [15]:
# количество классов
num_classes = len(train_df['Category'].unique())
num_classes

2

In [16]:
# получение путей изображений для тестовой выборки
test_dir = os.path.join('/content', 'test', 'test1')
test_images = os.listdir(test_dir)

In [17]:
# имена изображений тестовой выборки
test_images = test_images[:2500]

In [18]:
answers = pd.read_csv('sampleSubmission.csv')

In [19]:
test_df = pd.DataFrame({'Image': test_images, 'Category' : answers['label'][2500]})
test_df.head()

,Image,Category
0,5900.jpg,0
1,8226.jpg,0
2,435.jpg,0
3,4206.jpg,0
4,8833.jpg,0


In [20]:
# формирование тестовой выборки
test = []
for img_name, category in zip(test_df['Image'], test_df['Category']):
  img, x = get_image(os.path.join(test_dir, img_name))
  test.append({'x': np.array(x[0]), 'y' : category})

In [21]:
# разделения данных
X_train, y_train = np.array([t["x"] for t in train]), np.array([int(t["y"]) for t in train])
X_test, y_test= np.array([t["x"] for t in test]), np.array([int(t["y"]) for t in test])

In [22]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Работа с нейросетью

In [23]:
# нейросеть
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [24]:
# создание ссылки на входящий слой нейросети VGG
inp = vgg.input

# создание нового sigmoid слоя с 1 нейроном
new_classification_layer = Dense(num_classes - 1, activation='sigmoid')

# подсоединение нашего нового слоя к предпоследнему слою и создание ссылки на него
out = new_classification_layer(vgg.layers[-2].output)

# создание новой нейросети
model_new = Model(inp, out)

In [25]:
# информация о новой нейросети
model_new.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [26]:
# заморозка слоев
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

In [27]:
# компиляция модели
model_new.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [28]:
# обучение модели
model_new.fit(X_train, y_train, batch_size=10, epochs=10)

Epoch 1/10
250/250 [==============================] - 23s 56ms/step - loss: 0.5661 - accuracy: 0.6999
Epoch 2/10
250/250 [==============================] - 14s 57ms/step - loss: 0.3114 - accuracy: 0.8691
Epoch 3/10
250/250 [==============================] - 15s 60ms/step - loss: 0.2361 - accuracy: 0.9076
Epoch 4/10
250/250 [==============================] - 15s 61ms/step - loss: 0.2336 - accuracy: 0.9058
Epoch 5/10
250/250 [==============================] - 15s 61ms/step - loss: 0.2159 - accuracy: 0.9126
Epoch 6/10
250/250 [==============================] - 14s 58ms/step - loss: 0.2041 - accuracy: 0.9155
Epoch 7/10
250/250 [==============================] - 14s 57ms/step - loss: 0.2145 - accuracy: 0.9155
Epoch 8/10
250/250 [==============================] - 15s 58ms/step - loss: 0.1965 - accuracy: 0.9174
Epoch 9/10
250/250 [==============================] - 15s 59ms/step - loss: 0.1927 - accuracy: 0.9179
Epoch 10/10
250/250 [==============================] - 15s 60ms/step - loss: 0.193

# Оценка качества

In [29]:
# функция потерь, accuracy модели
loss, accuracy = model_new.evaluate(X_test, y_test)

print('Значение функции потерь: ', loss)
print('Значение accuracy', accuracy)

79/79 [==============================] - 19s 155ms/step - loss: 2.9890 - accuracy: 0.4696
Значение функции потерь:  2.989013671875
Значение accuracy 0.46959999203681946


In [30]:
# MSE модели
print('MSE: ', metrics.mean_absolute_error(y_test, model_new.predict(X_test)))

MSE:  0.5330229690720016
